In [1]:
from pathlib import Path
import numpy as np
from funcionesTP import *

path_dataset_train = '../TP/template-alumnos/dataset/cats_and_dogs/train'
path_dataset_val = '../TP/template-alumnos/dataset/cats_and_dogs/val'

def cargarDataset(carpeta: Path):
    """
    carpeta Path a la carpeta de train / val
    retorna la matriz de embeddings (1536x2000 / 1536x1000) y la matriz Y (2x2000 / 2x1000)
    """
    
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

def cargarDatasetCompleto():
    """
    retorna el dataset de entrenamiento (X_t, Y_t) y el de valuación (X_v, Y_v)
    """

    
    Xt, Yt = cargarDataset(Path(path_dataset_train))
    Xv, Yv = cargarDataset(Path(path_dataset_val))

    return Xt, Yt, Xv, Yv

Las siguientes funciones generan la matriz W a partir del calculo de la pseudoinversa de X utilizando diferentes metodos de decomposicion

In [2]:
def generar_W_EcuacionesNormales():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    W = pinvEcuacionesNormales(Xt, Yt)
    np.save('./W_eqnormales', W)

def generar_W_QRHH():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_HH(traspuesta(Xt))
    W = qrFCN(Q, R, Yt)

    np.save('./W_qrhh', W)

def generar_W_QRGS():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_GS(traspuesta(Xt)) # type: ignore

    W = qrFCN(Q, R, Yt)
    np.save('./W_qrgs', W)

def generar_W_SVD():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    W = svdFCN(Xt, Yt)
    np.save('./W_SVD', W)
    

Utilizando estas cuatro funciones, generamos la matriz de pesos *W* con los distintos métodos

In [3]:
generar_W_QRHH()

100%|██████████| 2/2 [00:00<00:00, 39.82it/s]


In [ ]:
def test_W(path_W, dataset = ['val', 'train']):
    """
    path_W Path al archivo de la matriz de pesos W generada con alguno de los métodos
    dataset las matrices que se van a usar para el test. Puede elegir entre entrenamiento o validación     

    utilizando la norma 2, calculamos la norma del residuo ||Y - WX||_2 para tener una medida error
    sobre el conjunto de validación.

    Retorna Norma 2 de la diferencia entre Y y W @ X
    """
    W = np.load(path_W)
    Xv, Yv = cargarDataset(Path(f'../TP/template-alumnos/dataset/cats_and_dogs/{dataset}'))

    matrizDiferencia = Yv - (W @ Xv)
    return np.linalg.norm(matrizDiferencia, ord=2)

In [ ]:
test_W('./W_qrhh.npy', 'val')

In [ ]:
# Evalúa el modelo contando cuántos gatos y perros fueron clasificados correctamente y cuántos incorrectamente.
def valoresDeConfusion(W_file):
    W = np.load(W_file)
    Xv, _ = cargarDataset(Path('../TP/template-alumnos/dataset/cats_and_dogs/val'))
    matriz_resultado = W@Xv 
    gatos_positivos = 0

    for i in range(500):
        resultado = matriz_resultado[:,i]
        if resultado[0] > resultado[1]:
            gatos_positivos += 1 
    
    perros_positivos = 0

    for i in range(500, 1000):
        resultado = matriz_resultado[:,i]
        if resultado[1] > resultado[0]:
            perros_positivos += 1

    return gatos_positivos, 500 - gatos_positivos, perros_positivos, 500 - perros_positivos          

valoresDeConfusion('./W_qrgs.npy')